In [2]:
use Modern::Perl;
use HackaMol;
use YAML::XS qw(Dump DumpFile);

###Prepare the working environment using a HackaMol object

In [3]:
my $bldr = HackaMol->new(data => "../xtals_clusters", scratch=>"results/xtals");
my @pdbs = $bldr->data->children(qr/\.pdb/);
$bldr->scratch->mkpath unless $bldr->scratch->exists;

1


###Add a couple of useful subroutines

In [4]:
sub CysCys_intcoords{
# Depends on convention of PDB ordering 
#0    N CYS     
#1   CA CYS    
#2    C CYS     
#3    O CYS     
#4   CB CYS    
#5   SG CYS    
#
  my $ss    = shift;
  my $sa    = $ss->get_atoms(0);
  my $sb    = $ss->get_atoms(1);

  my $q_intra = 0;
  $q_intra = abs($sa->resid - $sb->resid)-1  if ($sa->chain eq $sb->chain);

  my @atoms = @_;

  my @cysa  = grep{ 
                    $_->resid eq $sa->resid 
                and $_->chain eq $sa->chain
                  } @atoms;

  my @cysb  = grep{ 
                    $_->resid eq $sb->resid
                and $_->chain eq $sb->chain
                  } @atoms;

  return ("MISSING_ATOMS") if (scalar(@cysa) < 6 or
                               scalar(@cysb) < 6);  

  my $hack   = HackaMol->new(name=>"builder");

  my ($CACA)   = $hack->build_bonds(  $cysa[1], $cysb[1]  ) ;
  my ($CACBS1) = $hack->build_angles( @cysa[1,4,5]        ) ;
  my ($CACBS2) = $hack->build_angles( @cysb[1,4,5]        ) ;
  my ($CBS1S2) = $hack->build_angles( @cysa[4,5],$cysb[5] ) ;
  my ($CBS2S1) = $hack->build_angles( @cysb[4,5],$cysa[5] ) ;

  return (  
          { distance => {
              ss   => $ss->bond_length   ,
              caca => $CACA->bond_length , 
            },
            angle => {
              cacbs1 => $CACBS1->ang_deg   , 
              cacbs2 => $CACBS2->ang_deg   ,  
              cbs1s2 => $CBS1S2->ang_deg   , 
              cbs2s1 => $CBS2S1->ang_deg   , 
            },
            chain => dseq_inter_intra($sa,$sb),
          }
  );
}

In [5]:
sub dseq_inter_intra {
  my ($sa,$sb) = (shift,shift);
  my $dseq = abs($sa->resid - $sb->resid);
  my ($qinter,$qintra) = (0,1);
  ($qinter,$qintra)    = (1,0) unless ($sa->chain eq $sb->chain);
  return (
          {
           dseq   => $dseq,
           qinter => $qinter,
           qintra => $qintra,
          },
  );
}

In [6]:
my %coords = map {
  my $fpdb = $_;
  my $name = $fpdb->basename('.pdb');
  my $mol   = $bldr->read_file_mol($fpdb);
  # use Hg atom to analyze the disulfide that is actually of interest
  # eliminate double counting
  my ($hg)  = grep {$_->symbol eq 'Hg' } $mol->all_atoms;

  my ($ss)  = grep {abs($_->COM-$hg->xyz) <= 0.001} 
              $bldr->find_disulfide_bonds($mol->all_atoms);
  my @lcor;
  # for NMR ensemble of structures   
  foreach my $t (0 .. $mol->tmax){
    $mol->t($t);
    my @SScor = CysCys_intcoords($ss, $mol->all_atoms);
    push @lcor, @SScor unless (grep {/MISSING/} @SScor);
    #push @lcor, [@SScor] unless (grep {/MISSING/} @SScor);
  }
  ($name,\@lcor);
} @pdbs;

my $fyaml = $bldr->scratch->child("xtals\_distances_angles.yaml");
DumpFile($fyaml,\%coords);

1


In [8]:
use List::Util qw(max min sum);
use Math::SimpleHisto::XS;

In [9]:
my $nbin = 50;
my $min  = 1.95;
my $max  = 2.15;

my @ssb;
foreach my $cluster (keys %coords){
  foreach my $t (@{$coords{$cluster}}){
    push @ssb, $t->{distance}{ss};
  }
}

In [10]:
my $df = ($max-$min)/ $nbin;
my $hist = Math::SimpleHisto::XS->new(
          bins => [ map{ $min + $df*$_ } (0 .. $nbin)],
);

$hist->fill(\@ssb);
$hist->normalize($hist->total/scalar(@ssb));
my $data_bins   = $hist->all_bin_contents;
my $bin_centers = $hist->bin_centers;

my $sum = 0;
foreach my $i (0 .. $#{$data_bins}){
  $sum += $data_bins->[$i];
  printf ("%10.4f %10.4f\n", $bin_centers->[$i], $data_bins->[$i]);
}




say "\n\nsum: $sum\n";

    1.9520     0.0005
    1.9560     0.0007
    1.9600     0.0004
    1.9640     0.0009
    1.9680     0.0016
    1.9720     0.0007
    1.9760     0.0013
    1.9800     0.0014
    1.9840     0.0018
    1.9880     0.0026
    1.9920     0.0026
    1.9960     0.0037
    2.0000     0.0050
    2.0040     0.0097
    2.0080     0.0110
    2.0120     0.0125
    2.0160     0.0187
    2.0200     0.0320
    2.0240     0.0437
    2.0280     0.0801
    2.0320     0.1129
    2.0360     0.1029
    2.0400     0.0845
    2.0440     0.0685
    2.0480     0.0580
    2.0520     0.0454
    2.0560     0.0342
    2.0600     0.0295
    2.0640     0.0297
    2.0680     0.0200
    2.0720     0.0207
    2.0760     0.0185
    2.0800     0.0135
    2.0840     0.0129
    2.0880     0.0126
    2.0920     0.0097
    2.0960     0.0095
    2.1000     0.0079
    2.1040     0.0066
    2.1080     0.0058
    2.1120     0.0047
    2.1160     0.0045
    2.1200     0.0058
    2.1240     0.0045
    2.1280     0.0036
    2.1320

1


In [11]:
printf ("%10.4f \n", $bin_centers->[$_]) foreach 0 .. $#{$data_bins};$data_bins->[$i]

    1.9520 
    1.9560 
    1.9600 
    1.9640 
    1.9680 
    1.9720 
    1.9760 
    1.9800 
    1.9840 
    1.9880 
    1.9920 
    1.9960 
    2.0000 
    2.0040 
    2.0080 
    2.0120 
    2.0160 
    2.0200 
    2.0240 
    2.0280 
    2.0320 
    2.0360 
    2.0400 
    2.0440 
    2.0480 
    2.0520 
    2.0560 
    2.0600 
    2.0640 
    2.0680 
    2.0720 
    2.0760 
    2.0800 
    2.0840 
    2.0880 
    2.0920 
    2.0960 
    2.1000 
    2.1040 
    2.1080 
    2.1120 
    2.1160 
    2.1200 
    2.1240 
    2.1280 
    2.1320 
    2.1360 
    2.1400 
    2.1440 
    2.1480 


In [12]:
printf ("%10.4f \n", $data_bins->[$_]) foreach 0 .. $#{$data_bins};

    0.0005 
    0.0007 
    0.0004 
    0.0009 
    0.0016 
    0.0007 
    0.0013 
    0.0014 
    0.0018 
    0.0026 
    0.0026 
    0.0037 
    0.0050 
    0.0097 
    0.0110 
    0.0125 
    0.0187 
    0.0320 
    0.0437 
    0.0801 
    0.1129 
    0.1029 
    0.0845 
    0.0685 
    0.0580 
    0.0454 
    0.0342 
    0.0295 
    0.0297 
    0.0200 
    0.0207 
    0.0185 
    0.0135 
    0.0129 
    0.0126 
    0.0097 
    0.0095 
    0.0079 
    0.0066 
    0.0058 
    0.0047 
    0.0045 
    0.0058 
    0.0045 
    0.0036 
    0.0029 
    0.0024 
    0.0032 
    0.0020 
    0.0014 
